In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
import time
import scipy.linalg as spla

from lib.NR3 import NR3
from lib.helper import nominal_load_values, nominal_cap_arr
from lib.map_output import map_output
from lib import vvc
import sys

np.set_printoptions(threshold=sys.maxsize)

In [2]:
s = vvc.voltVARControl([0.92, 0.98, 1.02, 1.08], -45, 45)


In [3]:
print(s)

Hi!


TypeError: __str__ returned non-string (type NoneType)

In [ ]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

#fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = '05node_singlephase_balanced_oscillation_03.dss'
#fn = '06node_threephase_unbalanced.dss'
#fn = 'IEEE_37_Bus_allwye.dss'
#fn = 'IEEE_13_Bus_allwye_noxfm_noreg.dss'
#fn = 'IEEE_13_Bus_allwye_noxfm_noreg.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
fn = '06node_threephase_meshed_unbalanced.dss'
#fn = '06node_threephase_unbalanced_VR.dss'
#fn = '02node_threephase_unbalanced_new.dss'
#fn = '06node_threephase_radial_unbalanced_transformer_Copy.dss'
#fn = '06node_threephase_radial_unbalanced_regulator_simple.dss'
#fn = '06node_threephase_radial_unbalanced_regulator.dss'
#fn = 'IEEE_34_Bus_allwye.dss'
#fn = 'IEEE_37_Bus_allwye_noxfm_noreg_ffresh.dss' #not working

fn = 'IEEE_34_Bus_allwye_noxfm_noreg.dss' #not working

#fn = '06node_threephase_radial_unbalanced.dss'
fn = 'IEEE_37_Bus_allwye.dss'

#fn = 'IEEE_34_Bus_allwye_test_01.dss'
#fn = 'IEEE_13Node_Modified_01.dss'

# Time-steps
times = np.linspace(0, 2*np.pi, 5)
times = [0]

# DER changes over time-steps
der = [0, 0, 0, 0, 0]
der = [0]

# Capacitance changes over time-steps
capacitance = [0, 0, 0, 0, 0]
capacitance = [0]


t00 = time.time()

dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")


vvc1 = voltVARControl([0.92, 0.98, 1.02, 1.08], -45, 45)


for i in range(len(times)):
    # Run NR3 with variations in time, DER, and capacitance
    
    XNR = NR3(fn, slackidx, Vslack, None, None,None, None, der[i], capacitance[i], times[i])   
    VNR, INR, STXNR, SRXNR, iNR, sNR = map_output(nline, nnode, XNR, fn, times[i])
                    
    
    VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
    INR01[i, :, :] = np.reshape(INR, (3, nline))
    STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR01[i, :, :] = np.reshape(sNR, (3, nnode))
    
##################
    
# fn = 'IEEE_34_Bus_allwye_noxfm_noreg.dss'

# dss.run_command('Redirect ' + fn)
# nnode = len(dss.Circuit.AllBusNames())
# nline = len(dss.Lines.AllNames())

# VNR012 = np.zeros((len(times), 3, nnode), dtype = "complex")
# INR012 =  np.zeros((len(times), 3, nline), dtype = "complex")
# STXNR012 = np.zeros((len(times), 3, nline), dtype = "complex")
# SRXNR012 = np.zeros((len(times), 3, nline), dtype = "complex")
# iNR012 = np.zeros((len(times), 3, nnode), dtype = "complex")
# sNR012 = np.zeros((len(times), 3, nnode), dtype = "complex")

# for i in range(len(times)):
#     # Run NR3 with variations in time, DER, and capacitance
    
#     XNR2 = NR3(fn, slackidx, Vslack, None, None,None, None, der[i], capacitance[i], times[i])   
#     VNR2, INR2, STXNR2, SRXNR2, iNR2, sNR2 = map_output(nline, nnode, XNR2, fn, times[i])
                    
    
#     VNR012[i, :, :] = np.reshape(VNR2, (3, nnode))
#     INR012[i, :, :] = np.reshape(INR2, (3, nline))
#     STXNR012[i, :, :] = np.reshape(STXNR2, (3, nline))
#     SRXNR012[i, :, :] = np.reshape(SRXNR2, (3, nline))
#     iNR012[i, :, :] = np.reshape(iNR2, (3, nnode))
#     sNR012[i, :, :] = np.reshape(sNR2, (3, nnode))


In [ ]:
fn = 'IEEE_34_Bus_allwye.dss'
dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

In [ ]:


dss.run_command('Redirect ' + fn)
snrdf2 = pd.DataFrame(data=sNR,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
snrdf2 #IEEE 34

In [ ]:
fn = 'IEEE_34_Bus_allwye_noxfm_noreg.dss'
dss.run_command('Redirect ' + fn)
snrdf = pd.DataFrame(data=sNR2,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
snrdf #IEEE 34

In [ ]:
snrnew = snrdf2.drop(['852r', '814r'], axis=0)

In [ ]:
(np.abs(snrnew - snrdf)).sort_values(by='C')

In [ ]:
print(dss.Lines.AllNames())
print(dss.RegControls.AllNames())

In [ ]:
fn = 'IEEE_37_Bus_allwye.dss'
dss.run_command('Redirect ' + fn)

for vr in dss.RegControls.AllNames():
    dss.RegControls.Name(vr)
    print(dss.RegControls.Name())
    print(dss.RegControls.CTPrimary())
    print(dss.RegControls.ForwardX())
    print(dss.RegControls.ForwardVreg())
    print(dss.RegControls.Winding())
    print(dss.CktElement.BusNames())
    print(dss.RegControls.ForwardR(), "\n")
    
    
   
    #V_drop = (dss.RegControls.ForwardR() + 1j*dss.RegControls.ForwardX()) / 0.2 * (I_reg / dss.RegControls.CTPrimary())
    #V_r = dss.RegControls.ForwardVreg() - V_drop
    
    #absV_r = np.abs(V_r)
    
def voltage_regulator_index_dict():
    related_vr = {}
    for n in range(len(dss.RegControls.AllNames())):
        dss.RegControls.Name(dss.RegControls.AllNames()[n])
        dss.Circuit.SetActiveBus(dss.CktElement.BusNames()[0].split(".")[0])
        if dss.Bus.Name() in related_vr.keys():       
            related_vr[dss.Bus.Name()].append(n)
        else:        
            related_vr[dss.Bus.Name()] = [n] 
    return related_vr

voltage_regulator_index_dict()

In [ ]:
try:
    dss.RegControls.Hi()
    print('hi')
except:
    print('no')
print(len(dss.Transformers.AllNames()))
print(len(dss.RegControls.AllNames()))

OpenDSS Solution

In [ ]:
def get_dss_loads(times):

    dss_loads = np.zeros((len(times), 3, nnode), dtype='complex')
    dss_nom_loads = np.zeros((len(times), 3, nnode), dtype='complex')
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        load_phases = [0, 0, 0]
        load_data = dss.CktElement.BusNames()[0].split('.')[1:]
        idxbs = dss.Circuit.AllBusNames().index((dss.CktElement.BusNames()[0].split('.')[0])) #match busname to index
    
        for l in load_data:
            phase = int(l)
            load_phases[phase-1] = 1
            
        if len(load_data) == 0:
            load_phases = [1, 1, 1]  
        no_phases = np.sum(load_phases)
        n = 0
        for ph in range(len(load_phases)):
            if load_phases[ph] == 1:  
                dss_loads[0, ph, idxbs] += (dss.CktElement.Powers()[::2][n] + 1j * dss.CktElement.Powers()[1::2][n])*1e3/Sbase 
                dss_nom_loads[0, ph, idxbs] += (dss.Loads.kW() + 1j *dss.Loads.kvar())*1e3/Sbase/no_phases                   
                n += 1   
    return dss_loads, dss_nom_loads

def get_dss_loads_not_bus(times):
    nload = len(dss.Loads.AllNames())
    dss_loads = np.zeros((len(times), 3, nload), dtype='complex')
    dss_nom_loads = np.zeros((len(times), 3, nload), dtype='complex')
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        load_phases = [0, 0, 0]
        load_data = dss.CktElement.BusNames()[0].split('.')[1:]
        idxbs = dss.Circuit.AllBusNames().index((dss.CktElement.BusNames()[0].split('.')[0])) #match busname to index
    
        for l in load_data:
            load_phases[int(l)-1] = 1
            
        if len(load_data) == 0:
            load_phases = [1, 1, 1]
            
        no_phases = np.sum(load_phases)
        
        n = 0
        for ph in range(len(load_phases)):        
            if load_phases[ph] == 1:  
                
                dss_loads[0, ph, k] += (dss.CktElement.Powers()[::2][n] + 1j * dss.CktElement.Powers()[1::2][n])*1e3/Sbase 
                dss_nom_loads[0, ph, k] += (dss.Loads.kW() + 1j *dss.Loads.kvar())*1e3/Sbase/no_phases                   
                n += 1   
    return dss_loads, dss_nom_loads


def get_dss_capacitors(times):
    dss_capacitors = np.zeros((len(times), 3, nnode), dtype='complex')
    dss_nom_capacitors = np.zeros((len(times), 3, nnode), dtype='complex')

    for k in range(len(dss.Capacitors.AllNames())):
        dss.Capacitors.Name(dss.Capacitors.AllNames()[k])
        cap_data = dss.CktElement.BusNames()[0].split('.')
        idxbs = dss.Circuit.AllBusNames().index(cap_data[0])
        cap_phases = [0, 0, 0]
        
        for p in range(len(cap_data[1:])):
            cap_phases[int(cap_data[1:][p]) - 1] = 1
            
        if len(cap_phases) == 0:
            cap_phases = [1, 1, 1]  
        no_phases = np.sum(cap_phases)
        n=0
        for ph in range(len(cap_phases)): 
            if cap_phases[ph] == 1:
              
                dss_capacitors[0, ph, idxbs] += 1j*dss.CktElement.Powers()[1::2][n] * 1e3 / Sbase 
                dss_nom_capacitors[0, ph, idxbs] += 1j*dss.Capacitors.kvar() * 1e3 / Sbase / no_phases
                n+=1

    return dss_capacitors, dss_nom_capacitors

def get_zip_matrix():
    aPQ = np.zeros((3,nnode))
    aI = np.zeros((3,nnode))
    aZ = np.zeros((3,nnode))

    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
       
        load_data = dss.CktElement.BusNames()[0].split('.')[1:]
        idxbs = dss.Circuit.AllBusNames().index((dss.CktElement.BusNames()[0].split('.')[0])) #match busname to index
    
        for l in load_data:
            phase = int(l)
            aPQ[phase - 1, idxbs] = 0
            aI[phase - 1, idxbs] = 0
            aZ[phase - 1, idxbs] = 1
            
        if len(load_data) == 0:
            aPQ[:, idxbs] = 0
            aI[:, idxbs] = 0
            aZ[:, idxbs] = 1
    return aPQ, aI, aZ
    

In [ ]:
t0_dss = time.time()
for _ in range(1):
#     dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
#    dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
#     dss.run_command('Redirect IEEE_37_Bus_allwye.dss')
#     dss.run_command('Redirect 06node_threephase_radial_unbalanced_regulator.dss')
#     dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#     dss.run_command('Redirect 06node_threephase_unbalancedVR.dss')
    #dss.run_command('Redirect 02node_threephase_unbalanced_new.dss')
    dss.run_command('Redirect 06node_threephase_meshed_unbalanced.dss')
#     dss.run_command('Redirect 06node_threephase_radial_unbalanced_transformer_Copy.dss')
#     dss.run_command('Redirect 06node_threephase_radial_unbalanced.dss')
    dss.run_command('Redirect IEEE_37_Bus_allwye.dss')
    #dss.run_command('Redirect IEEE_34_Bus_allwye_noxfm_noreg.dss')
    #dss.run_command('Redirect IEEE_37_Bus_allwye_noxfm_noreg_ffresh.dss')
    #dss.run_command('Redirect IEEE_34_Bus_allwye_test_01.dss')
   # dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
    
    dss.Solution.Solve()
    Sbase = 1000000.0
    
    ####### HERE
    dss_loads_before = np.zeros((len(times), 3, nnode), dtype='complex')
    dss_cap_before = np.zeros((len(times), 3, nnode), dtype='complex')
    dss_nom_loads_before = np.zeros((len(times), 3, nnode), dtype='complex')
    dss_nom_cap_before = np.zeros((len(times), 3, nnode), dtype='complex')
    
    dss_loads_before, dss_nom_loads_before = get_dss_loads(times)
    dss_cap_before, dss_nom_cap_before = get_dss_capacitors(times)   
  
    zipv1 = [1, 0, 0, 1, 0, 0, 0.8] # constant impedance
    zipv = [0, 0, 1, 0, 0, 1, 0.8] # constant power
    zipv2 = [0.5, 0, 0.5, 0.5, 0, 0.5, 0.8] # 
    for load in dss.Loads.AllNames():
        dss.Loads.Name(load)
        dss.Loads.Model(8)
        dss.Loads.ZipV(zipv1) 
    
    VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
    IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
    SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
    STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
    dss_loads = np.zeros((len(times), 3, nnode), dtype='complex')
    dss_loads_after = np.zeros((len(times), 3, nnode), dtype='complex')
    kW_list = np.array([])
    kvar_list = np.array([])

#     # Store the loads
#     for k in range(len(dss.Loads.AllNames())):
#         dss.Loads.Name(dss.Loads.AllNames()[k])
#         kW_list = np.append(kW_list, dss.Loads.kW())
#         kvar_list = np.append(kvar_list, dss.Loads.kvar())

    for i in range(len(times)):
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
        Vbase = dss.Bus.kVBase() * 1000
        Sbase = 1000000.0
        Ibase = Sbase/Vbase
        Zbase = Vbase/Ibase
       

        # Set slack bus (sourcebus) voltage reference in p.u.
        SlackBusVoltage = 1.000
        dss.Vsources.PU(SlackBusVoltage)

#         #Time-varying load
#         for k in range(len(dss.Loads.AllNames())):
#             dss.Loads.Name(dss.Loads.AllNames()[k])
#             dss.Loads.kW(kW_list[k]* 0.8*(1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
#             dss.Loads.kvar(kvar_list[k] * 0.8*(1 + 0.1*np.sin(2*np.pi*0.01*times[i])))

        dss.Solution.Convergence(0.000000000001)

        # Solve power flow with OpenDSS file
        dss.Solution.Solve()
        if not dss.Solution.Converged:
            print('Initial Solution Not Converged. Check Model for Convergence')
        else:
            print('Initial Model Converged. Proceeding to Next Step.')
            #Doing this solve command is required for GridPV, that is why the monitors
            #go under a reset process
            dss.Monitors.ResetAll()

            # set solution Params
            #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
            dss.Solution.Mode(1)
            dss.Solution.Number(1)
            dss.Solution.StepSize(1)
            dss.Solution.ControlMode(-1)
            dss.Solution.MaxControlIterations(1000000)
            dss.Solution.MaxIterations(3000)
            # Easy process to get all names and count of loads, a trick to avoid
            # some more lines of code
            TotalLoads=dss.Loads.Count()
            AllLoadNames=dss.Loads.AllNames()
            print('OpenDSS Model Compliation Done.')
            print('Iterations: ', dss.Solution.Iterations())
            print('Tolerance: ', dss.Solution.Convergence())

        print('')

        # OpenDSS loads
        
        ####### HERE 
        dss_loads_zip = np.zeros((len(times), 3, nnode), dtype='complex')
        dss_cap_zip = np.zeros((len(times), 3, nnode), dtype='complex')
        dss_nom_loads_zip = np.zeros((len(times), 3, nnode), dtype='complex')
        dss_nom_cap_zip = np.zeros((len(times), 3, nnode), dtype='complex')
      
        dss_loads_zip, dss_nom_loads_zip = get_dss_loads(times)
        dss_cap_zip, dss_nom_cap_zip = get_dss_capacitors(times)
               
        print(dss.Solution.Converged())

        # Print number of buses, and bus names
        print(len(dss.Circuit.AllBusNames()))
        print(dss.Circuit.AllBusNames())

        # Print number of loads, and load names
        print(len(dss.Loads.AllNames()))
        print(dss.Loads.AllNames())

        print('')

        VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

        for k1 in range(len(dss.Circuit.AllBusNames())):

            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
            ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
            Vtemp = np.asarray(dss.Bus.PuVoltage())
            Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
            VDSS[ph,k1] = Vtemp    

        print('VDSS\n', np.round(VDSS,decimals=6))


        IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

        for k1 in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
        #     print(dss.Lines.AllNames()[k1])
            dss.Circuit.SetActiveBus(dss.CktElement.BusNames()[0].split(".")[0])
            tempVbase = dss.Bus.kVBase() * 1000 #LN 
            Ibase = Sbase / tempVbase
            print(dss.Lines.Name())
            print(Ibase, "\n")
     
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            if len(ph) == 0:
                ph = [0, 1, 2]
            Imn = np.asarray(dss.CktElement.Currents())/Ibase
        #     print(Imn)
            Imn = Imn[0:int(len(Imn)/2)]
        #     print(Imn)
            Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
        #     print(Imn)
            IDSS[ph,k1] = Imn
        #     print('')

        print('IDSS\n', np.round(IDSS,decimals=6))

        STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
        SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

        for k1 in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            if len(ph) == 0:
                ph = [0, 1, 2]
            Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)
       
            STXtemp = Sk[0:int(len(Sk)/2)]
            SRXtemp = Sk[int(len(Sk)/2):]
           
            STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
            SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])

            STXDSS[ph,k1] = STXtemp
            SRXDSS[ph,k1] = SRXtemp

        VDSS0[i, :, :] = VDSS
        IDSS0[i, :, :] = IDSS
        SRXDSS0[i, :, :] = SRXDSS
        STXDSS0[i, :, :] = STXDSS

        print('STXDSS\n', np.round(STXDSS,decimals=6))
        print('SRXDSS\n', np.round(SRXDSS,decimals=6))

        print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
        print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
        print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))


In [ ]:
# Output Network Comparison
print('Voltage')
diffV = VDSS - VNR
diffV_df = pd.DataFrame(data=diffV,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T

print('max voltage diff: \n', np.max(np.abs(diffV_df)))
print('abs voltage diff: \n', np.abs(diffV_df))
VDSS_df = pd.DataFrame(data=VDSS,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
print('\n DSS Voltage:', VDSS_df)
VNR_df = pd.DataFrame(data=VNR,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T


print('\n NR Voltage:', VNR_df)

print("----------------------------")
print('\n\n Current')
diffI_df = pd.DataFrame(data=IDSS-INR,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T
IDSS_df =  pd.DataFrame(data=IDSS,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T
INR_df = pd.DataFrame(data=INR,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T

print('max current diff: \n', np.max(np.abs(diffI_df)))
print('abs current diff: \n', np.abs(diffI_df))
print('DSS Current:', IDSS_df)
print('NR Current:', INR_df)
print("----------------------------")

print('\n\n STXNR - sending end line power')
diffSTX = STXDSS - STXNR
diffSTX_df = pd.DataFrame(data=diffSTX,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T
STXDSS_df = pd.DataFrame(data=STXDSS,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T
STXNR_df = pd.DataFrame(data=STXNR,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T

print('max STX diff: \n', np.max(np.abs(diffSTX_df)))
print('abs STX diff: \n', np.abs(diffSTX_df))
print('DSS STX:', STXDSS_df)
print('NR STX:', STXNR_df)

print("----------------------------")
diffSRX = SRXDSS - SRXNR
diffSRX_df = pd.DataFrame(data=diffSRX,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T
SRXDSS_df = pd.DataFrame(data=SRXDSS,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T
SRXNR_df = pd.DataFrame(data=SRXNR,columns=[dss.Lines.AllNames()],index=['A', 'B', 'C']).T

print('max SRX diff: \n', np.max(np.abs(diffSRX_df)))
print('abs SRX diff: \n', np.abs(diffSRX_df))
print('DSS SRX:', SRXDSS_df)
print('NR SRX:', SRXNR_df)
print("----------------------------")
dss_total_load = dss_loads_zip +  dss_cap_zip

diffload = dss_total_load[0] - sNR

diffload_df = pd.DataFrame(data=diffload,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
sDSS_df = pd.DataFrame(data=dss_total_load[0],columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
sNR_df = pd.DataFrame(data=sNR,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T


print('\n\n sNR - total node powers')
print('load max diff', np.max(np.abs(diffload_df)))
print('load abs diff', np.abs(diffload_df))
print('\n DSS load:', sDSS_df)
print('\n NR load:', sNR_df)


# # print('\n\n iNR')
# # print('max current diff: \n', np.max(np.abs(SRXDSS - SRXNR)))
# # print('abs current diff: \n', np.abs(SRXDSS-SRXNR))



In [ ]:
# Nominal before ZIP
# A1 - DSS using kw/kvar
dss_nom_total_load_before =  dss_nom_loads_before - dss_nom_cap_before

dss_nom_total_load_before_df = pd.DataFrame(data=dss_nom_total_load_before[0],columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
print('dss nominal load before zip \n', dss_nom_total_load_before_df)

# A2 - NR3 using kw/kvar
nr3_nom_kw, nr3_nom_kvar = nominal_load_values(-1) #+, +
nr3_nom_cap = nominal_cap_arr() #+

nr3_nom_total_load = nr3_nom_kw + 1j * (nr3_nom_kvar - nr3_nom_cap)
nr3_nom_total_load_df = pd.DataFrame(data=nr3_nom_total_load,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
print('nr3 nominal load before zip \n', nr3_nom_total_load_df)

# Comparison
print('difference in nominal load before zip \n', (nr3_nom_total_load_df - dss_nom_total_load_before_df))
print(np.max(nr3_nom_total_load_df - dss_nom_total_load_before_df))

In [ ]:
# cktelement.powers after ZIP
#### B1 - DSS
dss_total_load = dss_loads_zip +  dss_cap_zip
dss_total_load_df = pd.DataFrame(data=dss_total_load[0],columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
print('B1 by bus', dss_total_load_df)

#### B2 - DSS
####b2 = dss.loads.kw and dss.loads.kvar and zip values and dss voltage and calculating everything ourselves
#b2 and b1 should match , snr with dss voltage-> solution of DSS 

aPQ, aI, aZ  = get_zip_matrix()
DSS_sNR = dss_nom_loads_before*(aPQ + aI*np.abs(VDSS) + aZ*np.abs(VDSS)**2) - dss_nom_cap_before;

sNR_DSS_df= pd.DataFrame(data=DSS_sNR[0],columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T
print('\n B2 by bus', sNR_DSS_df)
print('\n B2-B1 by bus' , sNR_DSS_df - dss_total_load_df, "\n\n\n max diff B2-B1 by bus", np.max( sNR_DSS_df - dss_total_load_df))

#### B3 - solved for by NR3
nr3_total_load_df = pd.DataFrame(data=sNR,columns=[dss.Circuit.AllBusNames()],index=['A', 'B', 'C']).T


# print('\n nr3', nr3_total_load_df)
# print('\n diff', dss_total_load_df - nr3_total_load_df)
# print('\n max diff',np.max(dss_total_load_df - nr3_total_load_df))


In [ ]:
aPQ, aI, aZ  = get_zip_matrix()
DSS_sNR = dss_nom_loads_before*(aPQ + aI*np.abs(VDSS) + aZ*np.abs(VDSS)**2) - 1j*dss_nom_cap_before.imag;
dss_total_load = dss_loads_zip + dss_cap_zip
#B1
dss_loads, dss_nom_loads = get_dss_loads_not_bus(times)
dss_total_by_load_df = pd.DataFrame(data=dss_loads[0],columns=[dss.Loads.AllNames()],index=['A', 'B', 'C']).T

dss_adj_load = np.zeros((3, len(dss.Loads.AllNames())),dtype='complex')

notnom, nom = get_dss_loads_not_bus(times)

#B2
for ph in range(3):
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        idxbs = dss.Circuit.AllBusNames().index((dss.CktElement.BusNames()[0].split('.')[0])) #match busname to index
        V = VDSS[ph,idxbs]       
        dss_adj_load[ph, k] = nom[0,ph, k]*np.abs(V)**2*aZ[ph, idxbs] -1j*dss_nom_cap_before[0,ph,idxbs].imag
dss_adj_load_df = pd.DataFrame(data=dss_adj_load,columns=[dss.Loads.AllNames()],index=['A', 'B', 'C']).T

print('B2 by load',dss_adj_load_df)
print('\n B1 by load', dss_total_by_load_df)
print('\n B2-B1 by load, pu difference', dss_adj_load_df - dss_total_by_load_df)
print("\n", np.max(dss_adj_load_df - dss_total_by_load_df))